In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark

     |████████████████████████████████| 217.8MB 52kB/s 
     |████████████████████████████████| 204kB 42.5MB/s 


In [0]:
import pyspark, os
from pyspark import SparkConf, SparkContext
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"

#connects our python driver to a local Spark JVM running on the Google Colab server virtual machine
try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass


#also include this short helper function for use later in this lab
def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
    print(x)


# Question 1A

In [4]:
def pair_sort(sub_list):
  sub_list = sub_list.strip().split()
  print(sub_list)
  new_list = []
  for index, i in enumerate(sub_list):
    for index2, j in enumerate(sub_list):
      if index2 < index:
        new_list.append(tuple(sorted((i, j))))
  return new_list

  # enumerate 1 , 2 , 3 ,4 

# print(pair_sort('FRO11987 ELE17451 ELE89019 SNA90258 GRO99222'))

file = sc.textFile("browsing.txt")
print(type(file))
# dbg(file)
pairs_of_cart = file.flatMap(lambda line: (pair_sort(line)))
pairs_of_cart = pairs_of_cart.map(lambda x: (x, 1))
counts = pairs_of_cart.reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], False) 
counts.take(5)

<class 'pyspark.rdd.RDD'>


[(('DAI62779', 'ELE17451'), 1592),
 (('FRO40251', 'SNA80324'), 1412),
 (('DAI75645', 'FRO40251'), 1254),
 (('FRO40251', 'GRO85051'), 1213),
 (('DAI62779', 'GRO73461'), 1139)]

#Question 1B

In [0]:
file = sc.textFile("browsing.txt") # RDD of the whole file
dbg(file)
def pair_sort_version2(sub_list):
  sub_list = sub_list.strip().split()
  new_list = []
  for index, i in enumerate(sub_list):
    for index2, j in enumerate(sub_list):
       if index2 < index:
          print(sub_list)
          if (i in broadcastVar.value and j in broadcastVar.value):
            new_list.append(tuple(sorted((i, j))))
  return new_list

  # def pair_sort(sub_list):
  # sub_list = sub_list.strip().split()
  # print(sub_list)
  # new_list = []
  # for index, i in enumerate(sub_list):
  #   for index2, j in enumerate(sub_list):
  #     if index2 < index:
  #       new_list.append(tuple(sorted((i, j))))
  # return new_list



all_items = file.flatMap(lambda line: [(item, 1) for item in line.strip().split()])

count_all_items = all_items.reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], False) 

greater_than_100 = count_all_items.filter(lambda x: x[1] >= 100)

broadcastVar = sc.broadcast(greater_than_100.collectAsMap()) #broadcast with only the strings

# print("Here")
dbg(broadcastVar.value)

pairs_of_cart = file.flatMap(lambda line: (pair_sort_version2(line)))
dbg(pairs_of_cart.count())
pairs_of_cart = pairs_of_cart.map(lambda x: (x, 1))
counts = pairs_of_cart.reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], False) 
dbg(counts.count())




['FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 ', 'GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 ', 'ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 ', 'ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 ', 'ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 ', 'ELE17451 GRO73461 DAI22896 SNA99873 FRO18919 DAI50921 SNA80192 GRO75578 ', 'ELE17451 ELE59935 FRO18919 ELE23393 SNA80192 SNA85662 SNA91554 DAI22177 ', 'ELE17451 SNA69641 FRO18919 SNA90258 ELE28573 ELE11375 DAI14125 FRO78087 ', 'ELE17451 GRO73461 DAI22896 SNA80192 SNA85662 SNA90258 DAI46755 FRO81176 ELE66810 DAI49199 DAI91535 GRO94758 ELE94711 DAI22177 ', 'ELE17451 SNA69641 DAI91535 GRO94758 GRO99222 FRO76833 FRO81176 SNA80192 DAI54690 ELE37798 GRO56989 ', 'ELE17451 GRO73461 DAI22896 GRO99222 SNA47306 GRO36567 ELE82555 SNA17715 SNA94781 DAI87514 GRO48282 GRO12935 SNA55952 DAI93692 DAI92253 FRO82427 ELE26917 DAI22177 ', 'GRO99222 DAI48891 

In [0]:
dbg(counts.count())


149099


#Question 2A


In [0]:
#Confidence, conf(I -> j) = support(I u j) / support(I)
#file is out baskets
#support I
#xy = counts() this is out baskets 
#total texts they are in = sum 
# support(I u j) = counts 
#support of i 

#208
#all items greater than 100 = greater_than_100 = support of i
#('GRO85051', 'FRO40251') = 1213,   GRO85051 in greater_than_100 = 1214  ('FRO40251', 'GRO85051')
#total = broadcastVar.value['DAI62779'] #this will return the value we need it acts like a dict

def division(ele1, ele2, num):
  total = broadcastVar.value[ele1]
  return(((ele1, ele2), num/total)) #total is the value held in the greater_than_100 broadcast

right_basket = counts.map(lambda x: division(x[0][0], x[0][1], x[1]))
left_basket = counts.map(lambda x: division(x[0][1], x[0][0], x[1]))
confidence = right_basket.union(left_basket).sortBy(lambda x: x[1], False) 
dbg(confidence.count())

298198


#Question 2B

In [0]:
#probability =
total_basket_count = file.map(lambda x:x.split(" ,")).count() # gets the total count of baskets
#number of baskets 31101

def calculate(tuple_names, num):
  ele1 = tuple_names[0][1]
  return((tuple_names[0], num - (broadcastVar.value[ele1]/total_basket_count)))

#interst score = 
interest_score = confidence.map(lambda x: calculate(x, x[1])).sortBy(lambda x: x[1], False) 
dbg(interest_score.take(5))

# total_sum = greater_than_100.map(lambda x: x[1]).count()
# dbg(total_sum)
# k = greater_than_100.filter(lambda x : x[0]=="SNA82528").collect()[0]
# dbg(j)
# dbg(k)

[(('DAI43868', 'SNA82528'), 0.9538739086342057), (('DAI93865', 'FRO40251'), 0.8752130156586605), (('GRO85051', 'FRO40251'), 0.8743892924296655), (('GRO38636', 'FRO40251'), 0.8658672212661371), (('ELE12951', 'FRO40251'), 0.8657790533945096)]


looking for 0.1247869843413395‬ for DAI93865', 'FRO40251'
31101 with origional buckets


